# RAG Architecture
I created a basic RAG architecture which suits our use case. I followed this amazing [tutorial](https://python.langchain.com/docs/use_cases/question_answering/quickstart) and adapted it to our use case and increased transparency. You need to install the requirements with the following command to be able to execute the RAG:
```
pip install langchain    
pip install langchain-community
pip install langchainhub
pip install langchain-openai
pip install chromadb bs4

```

Further you will need an open AI key. I suggest Pierre creates his own openAI key, then you can handle the payment when I am away on your own. Just follow this [OpenAI-Quickstart](
https://platform.openai.com/docs/quickstart?context=python), first register an account and then add the openAI key to your path.

In [2]:
# put in open ai key manually if you havent added it to the path already
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [18]:
# imports
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate

In [34]:
# Load, chunk and index the contents of suitable websites
loader = WebBaseLoader(
    web_paths=("https://asylumineurope.org/reports/country/germany/reception-conditions/housing/types-accommodation/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("left")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    """
    You are an assistant for helping refugees to get settled in Germany and give them individual guidenace. Use the following pieces of retrieved context to answer the question and ask follow-up questions. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Make sure to give a personal experience by asking questions about the personal situation to provide better answers for the next question.
    Question: {question} 
    Context: {context} 
    Answer:
    """
)


llm = ChatOpenAI(model_name="gpt-4", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
# asking a question. This method would need to be called from the frontend
rag_chain.invoke("What type of accomodation do I get?")

'In Germany, asylum seekers are generally provided with three types of accommodation: initial reception centres, which include arrival centres, special reception centres, and AnkER-centres; collective accommodation centres; and decentralised accommodation. The type of accommodation you get depends on your individual circumstances. Can you tell me more about your current situation so I can provide more specific information?'

In [16]:
# cleanup
vectorstore.delete_collection()